In [37]:
# I will use this cell to import every library that i need.
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns 

In [38]:
# in order to import the data, I need to know what kind of encoding my file has.

with open('data/attacks.csv') as attackscsv:
    print(attackscsv)

<_io.TextIOWrapper name='data/attacks.csv' mode='r' encoding='cp1252'>


In [39]:
# I open the file and check the headings.

attacks = pd.read_csv('data/attacks.csv', encoding='cp1252')
attacks.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [40]:
# I will check basic info and then  drop all the rows or columns that have no values or only 1.
def BasicCleanig (dataframe,NumberOfValuesInRow,NumberOfValuesInColum):
    '''This function recevies a DataFrame, the minimum number of values that I want in my rows and columns.
        It deletes the empty rows, the empty columns, and the duplicates.
        Returns the DataFrame with the changes aplied.'''
    dataframe.dropna(axis=1, thresh= NumberOfValuesInColum, inplace=True)
    dataframe.dropna(axis=0, thresh=NumberOfValuesInRow, inplace=True)
    dataframe.drop_duplicates(inplace=True)
    return dataframe

attacks = BasicCleanig(attacks,5,5)
print(attacks)

     Case Number         Date    Year        Type             Country  \
0     2018.06.25  25-Jun-2018  2018.0     Boating                 USA   
1     2018.06.18  18-Jun-2018  2018.0  Unprovoked                 USA   
2     2018.06.09  09-Jun-2018  2018.0     Invalid                 USA   
3     2018.06.08  08-Jun-2018  2018.0  Unprovoked           AUSTRALIA   
4     2018.06.04  04-Jun-2018  2018.0    Provoked              MEXICO   
...          ...          ...     ...         ...                 ...   
6297     ND.0005  Before 1903     0.0  Unprovoked           AUSTRALIA   
6298     ND.0004  Before 1903     0.0  Unprovoked           AUSTRALIA   
6299     ND.0003    1900-1905     0.0  Unprovoked                 USA   
6300     ND.0002    1883-1889     0.0  Unprovoked              PANAMA   
6301     ND.0001    1845-1853     0.0  Unprovoked  CEYLON (SRI LANKA)   

                   Area                             Location      Activity  \
0            California          Oceanside, S

In [ ]:
# The case number has the correct date, but in a weird format. I will get it with regex. Using (\d{4})\.(\d{2})\.(\d{2})



object
